In [43]:
import tensorflow as tf
#We will explicitly create a grapg inseat of default graph.
graph=tf.Graph()

In [44]:
#Now, we will set our new graph as the default while we construct our mode.

with graph.as_default():
    
    with tf.name_scope("variables"):
        
        #Variable to keep track of how many times the graph has been run.
        global_step =tf.Variable(0,dtype=tf.int32, trainable=False, name="global_step")
        #Variable that keeps track of the sum of all output values over time:
        total_output =tf.Variable(0,dtype=tf.float32, trainable=False, name="total_output")
        #Note: Reason for setting the parameter training=False is because it wont impact the model as we arent training 
        #anything.
        
    #Now we'll create a new core model named 'transformation' and will have 
    #seperate scope named "input","intermidiate_layer","output"
    
    with tf.name_scope("transformation"):
        
        #Seperate input layer
        with tf.name_scope("input"):
            #Create input placeholder that inputs a vector.
            a=tf.placeholder(tf.float32,shape=[None],name="input_placeholder_a")
            #Note: shape =[None] is used in-order to accept vecotr of any length.
        #Seperate middle layer
        
        with tf.name_scope("intermidiate_layer"):
            b=tf.reduce_prod(a,name="product_b")
            c=tf.reduce_sum(a,name="sum_c")
            #Note: tf.reduce_prod() will perform operation of multiplicxation on the whole vector whereas tf.multipy() 
            # will just accept 2 scalar values, hence tf.reduce_prod() is used in this case.
            
        #Seperate output layer
        with tf.name_scope("output"):
            output=tf.add(b,c,name="output")
            
        #Now we will update our global varible which were defined previously.
        
        with tf.name_scope("update"):
            
            #Increment the total_output Variable by latest output.
            update_total=total_output.assign_add(output)
            
            #Increment global_step variable: Will run whenever gragh runs, basically a counter.
            increment_step= global_step.assign_add(1)
            
        #We will create tensorBoard summaries
        
        with tf.name_scope("summaries"):
            avg=tf.div(update_total,tf.cast(increment_step,tf.float32),name="average")
            
            #Create Summaries for output node
            tf.summary.scalar("output_summary",output) 
            tf.summary.scalar("total_summary",update_total)
            tf.summary.scalar("average_summary",output)
            
        #Variable Initialization Operation
        with tf.name_scope("global_ops"):
            #Initialization Op
            init = tf.global_variables_initializer()
            #Merge all summaries into one Operation
            merged_summaries= tf.summary.merge_all()
    

In [45]:
#Running Graph 

sess=tf.Session(graph=graph)   
writer =tf.summary.FileWriter('./improved_graph',graph)       #tf.train.SummaryWriter
#Initialize all variables
sess.run(init)

In [54]:

def run_graph(input_tensor):
    """
    Helper function; runs the graph with given input tensor and saves summaries
    """
    feed_dict = {a: input_tensor}
    _,step,summary  = sess.run([output,increment_step,merged_summaries], feed_dict=feed_dict) # '_' is used in-order not ignore the sorting of output. 
    writer.add_summary(summary, global_step=step)




In [55]:
# Run the graph with various inputs
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])



In [56]:
# Writes the summaries to disk
writer.flush()

# Flushes the summaries to disk and closes the SummaryWriter
writer.close()

# Close the session
sess.close()

# To start TensorBoard after running this file, execute the following command:
# $ tensorboard --logdir='./improved_graph'

SyntaxError: can't assign to operator (<ipython-input-58-5b8bd0524c84>, line 1)